## Imports

In [38]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline  

from sklearn import svm, linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '../../../Utils/')
sys.path.insert(0, '../../../')
import cyphercat as cc

import models
from train import *
from metrics import * 
from SVC_Utils import *

#audio
import librosa as libr

print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Python: 3.6.5 (default, Jul  6 2018, 19:12:46) 
[GCC 5.4.0 20160609]
Pytorch: 0.4.0


## NN Hyperparameters

In [39]:
batch_size = 128
lr = 0.001
k = 3

pretrained = False #run this with networks that have already been trained

transform_type = 'SFTF' #either STFT or MFCC  

data = 'VOiCES' #'Libri' or 'VOiCES'

In [40]:
### Speech preprocessing

class tensorToMFCC:
    def __call__(self, y):
#         y = y.numpy()
        dims = y.shape
        y = libr.feature.melspectrogram(np.reshape(y, (dims[1],)), 16000, n_mels=number_of_mels,
                               fmax=8000)
        y = libr.feature.mfcc(S = libr.power_to_db(y))
        y = torch.from_numpy(y)                           
        return y.float()

class STFT:
    def __call__(self,y):
        dims = y.shape
        y = np.abs(libr.core.stft(np.reshape(y, (dims[1],))))
        y = torch.from_numpy(y).permute(1,0)
        return y.float()

if transform_type == 'SFTF':
    target_net_type = cc.ft_cnn_classifer
    shadow_net_type = cc.ft_cnn_classifer
    in_size = 94# 20 forMFCC,  94 for STFT
    transform  = STFT() ## STFT or MFCC
elif transform_type == 'MFCC':
    transform  = tensorToMFCC()
    target_net_type = cc.MFCC_cnn_classifier
    shadow_net_type = cc.MFCC_cnn_classifier
    in_size = 20

## Audio hyperparameters

In [41]:
n_seconds = 3
n_epochs = 15
sampling_rate = 16000
number_of_mels =128
lr = 0.001

# attacking means data for a target & shadow network.
# This will also split "out data" from totally different speakers -- data none of the 
# other networks have seen, for training & testing the attack network. This will be
# an equivalent amount of data to the train split as defined about

%load_ext autoreload
%autoreload 2
sys.path.insert(0, './../../../Utils')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load audio data: VOiCES or LibriSpeech, & Split into valid sequences

In [ ]:
print('Loading splits')

if data == 'Libri':
    dfs = cc.Libri_preload_and_split()
    print('Initializing dataset')
    valid_sequence_train_target = cc.LibriSpeechDataset(df=dfs[0], transform = transform)
    valid_sequence_test_target = cc.LibriSpeechDataset(df=dfs[1], transform = transform)
    valid_sequence_train_shadow = cc.LibriSpeechDataset(df=dfs[2], transform = transform)
    valid_sequence_test_shadow = cc.LibriSpeechDataset(df=dfs[3], transform = transform)
    valid_sequence_test = cc.LibriSpeechDataset(df=dfs[4], transform = transform)

    print('Succesfully loaded libri-speech')
elif data == 'VOiCES':
    dfs = cc.Voices_preload_and_split(split_type = 'segment')
    print('Initializing dataset')
    valid_sequence_train_target = cc.Voices_dataset(df=dfs[0], transform = transform)
    valid_sequence_test_target = cc.Voices_dataset(df=dfs[1], transform = transform)
#     valid_sequence_train_shadow = cc.Voices_dataset(df=dfs[2], transform = transform)
#     valid_sequence_test_shadow = cc.Voices_dataset(df=dfs[3], transform = transform)
#     valid_sequence_test = cc.Voices_dataset(df=dfs[4], transform = transform)
    print('Succesfully loaded VOiCES')

[autoreload of cyphercat.datadefs.voices_dataset failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/usr/local/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/local/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "../../../cyphercat/datadefs/voices_dataset.py", l

Loading splits
Initialising VOiCESDataset with minimum length = 3s and subset = room-2
Finished indexing data. 123888 usable files found.
Merging split 0
 Male: 15288 and Female: 15192
Merging split 1
 Male: 15288 and Female: 15192


In [35]:
# to look at the index file:

#look at splits file for reference
dff = pd.read_csv(os.getcwd()+'/../../../Datasets/splits/libri-train-clean-100/libri_4.csv')
dff.head()

df = pd.read_csv(os.getcwd()+'/../../../Datasets/VOiCES-room-2.index2.csv')
df.head()

g = df.groupby(['id','Section']).groups

dfn = pd.DataFrame(columns = ['id','Section'])
idx = 0
for key in g.keys():
    dfn.at[idx,'id']=key[0]
    dfn.at[idx,'Section']=key[1]
    idx +=1

print(len(np.unique(df.id)), 'speakers')
print(len(df), 'files')
print(dfn.groupby('id').count().min()[0], 'tracks min')
print(dfn.groupby('id').count().max()[0], 'tracks max')
print(dfn.groupby('id').count().mean()[0], 'tracks mean')

print('min speaker minutes',df.groupby('id').mean()['speaker_minutes'].min())
print('max speaker minutes',df.groupby('id').mean()['speaker_minutes'].max())
print('mean speaker minutes',df.groupby('id').mean()['speaker_minutes'].mean())

200 speakers
123984 files
9 tracks min
17 tracks max
11.35 tracks mean
min speaker minutes 146.0639999999989
max speaker minutes 260.18799999999635
mean speaker minutes 161.39901224999986


In [37]:
df.at[0,'filepath']

'VOiCES/room-2/Volumes/Lab41-SRI-VOiCES/VOiCES_distant-16k_speech/rm2/tele/sp_0032-1182/sp0032/Lab41-SRI-VOiCES-rm2-tele-sp0032-ch021625-sg0007-mc06-lav-far-dg120.wav'

In [7]:
batch_size = 32


# Loaders for data for target model & shadow model 
target_train_loader = DataLoader(valid_sequence_train_target,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )


target_test_loader = DataLoader(valid_sequence_test_target,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )

shadow_train_loader = DataLoader(valid_sequence_train_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )


shadow_test_loader = DataLoader(valid_sequence_test_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


test_loader = DataLoader(valid_sequence_test,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


## Set up model tracking

In [8]:
#table with summary

# To do: extract accuracy from train/eval funcs and automatically add to table
summary_file = 'summary.pkl'
columns = ['Transform','Data','Training epochs', '# speakers','Train accuracy', 'Test accuracy', 'Attack type', 'Precision','Recall']

try:
    df = pd.read_pickle(summary_file)

except:
    df = pd.DataFrame(columns = columns)
    
df_idx = len(df)

#set a bunch of known values
df.at[df_idx,'Transform'] =transform_type
df.at[df_idx,'Training epochs'] = n_epochs
df.at[df_idx,'Data'] = data
df.at[df_idx,'Attack type'] = 1

# Initialize/Train Targets
The model being attacked; if network, architecture can differ from that of shadow network.

In [9]:
#Initialize NN

#in_size defined above
n_hidden = 512
n_classes = valid_sequence_test_target.num_speakers
print(n_classes,' speakers')
df.at[df_idx,'# speakers']=n_classes


target_net = target_net_type(n_classes).to(device)
target_net.apply(models.weights_init)

target_loss = nn.CrossEntropyLoss()
target_optim = optim.Adam(target_net.parameters(), lr=lr)

100  speakers


In [11]:
#file name for this set of hyperparameters
fn = 'model_weights/CNN_voice_classifier'+data+'_target_'+transform_type+str(n_epochs-1)+'.pth'

#Train NN
if not pretrained:
    train_accuracy, test_accuracy = cc.train(target_net, target_train_loader, target_test_loader, target_optim, target_loss, n_epochs, verbose = False) 
    df.at[df_idx,'Train accuracy'] =round(train_accuracy,4)
    df.at[df_idx,'Test accuracy'] = round(test_accuracy,4)
    cc.save_checkpoint(model = target_net, optimizer = target_optim,
                           epoch = n_epochs-1, data_descriptor = data, 
                           accuracy = [train_accuracy, test_accuracy],
                           filename = fn)
    
else:
    cc.load_checkpoint(model = target_net, optimizer = target_optim, checkpoint = fn)


[0/15]
Training:

Accuracy = 65.03 %%


Test:

Accuracy = 66.04 %%


[1/15]
Training:

Accuracy = 79.01 %%


Test:

Accuracy = 80.69 %%


[2/15]
Training:

Accuracy = 83.04 %%


Test:

Accuracy = 84.81 %%


[3/15]
Training:

Accuracy = 84.20 %%


Test:

Accuracy = 86.48 %%


[4/15]
Training:

Accuracy = 86.94 %%


Test:

Accuracy = 87.68 %%


[5/15]
Training:

Accuracy = 87.59 %%


Test:

Accuracy = 89.57 %%


[6/15]
Training:

Accuracy = 88.36 %%


Test:

Accuracy = 89.33 %%


[7/15]
Training:

Accuracy = 92.54 %%


Test:

Accuracy = 93.49 %%


[8/15]
Training:

Accuracy = 91.38 %%


Test:

Accuracy = 92.33 %%


[9/15]
Training:

Accuracy = 93.12 %%


Test:

Accuracy = 93.55 %%


[10/15]
Training:

Accuracy = 91.81 %%


Test:

Accuracy = 93.07 %%


[11/15]
Training:

Accuracy = 94.26 %%


Test:

Accuracy = 94.80 %%


[12/15]
Training:

Accuracy = 93.23 %%


Test:

Accuracy = 93.75 %%


[13/15]
Training:

Accuracy = 94.77 %%


Test:

Accuracy = 95.44 %%


[14/15]
Training:

Accuracy = 

# Initialize/Train Shadow Model
Shadow model mimics the target network, emulating the target model's differences in prediction probabilities for samples in and out of its dataset. For this attack, only one shadow model is used. 

In [12]:
#Initialize models

n_classes = valid_sequence_test_shadow.num_speakers
print('n shadow speakers',n_classes)

#NN
shadow_net = shadow_net_type(n_classes).to(device)
shadow_net.apply(models.weights_init)

shadow_loss = nn.CrossEntropyLoss()
shadow_optim = optim.Adam(shadow_net.parameters(), lr=lr)
shadow_epochs = 1 #n_epochs

n shadow speakers 100


# Initialize Attack Model
A binary classifier to determine membership. 

In [13]:
# Attack the network: 

attack_net_nn = models.mlleaks_mlp(n_in=k).to(device)
attack_loss = nn.BCEWithLogitsLoss()
attack_optim_nn= optim.Adam(attack_net_nn.parameters(), lr=lr)
n_epochs_attack = 1

df_pr = cc.ml_leaks1(target=target_net, shadow_model = shadow_net, attacker_model = attack_net_nn,
            target_in_loader = test_loader, target_out_loader = target_test_loader,
            shadow_train_loader = shadow_train_loader, shadow_out_loader=shadow_test_loader,
            shadow_optim = shadow_optim, attack_optim = attack_optim_nn, 
            shadow_criterion = shadow_loss, attack_criterion = attack_loss, 
            shadow_epochs = shadow_epochs, attack_epochs = n_epochs_attack, retrain = True)

---- Training shadow network ----
[0/1]
Training:

Accuracy = 57.55 %%


Test:

Accuracy = 56.12 %%


---- Training attack network ----
[0/1][0/952] loss = 0.71, accuracy = 50.00
[0/1][1/952] loss = 0.71, accuracy = 50.00
[0/1][2/952] loss = 0.70, accuracy = 50.00
[0/1][3/952] loss = 0.70, accuracy = 50.00
[0/1][4/952] loss = 0.70, accuracy = 50.00
[0/1][5/952] loss = 0.70, accuracy = 50.00
[0/1][6/952] loss = 0.70, accuracy = 50.00
[0/1][7/952] loss = 0.70, accuracy = 50.00
[0/1][8/952] loss = 0.69, accuracy = 50.00
[0/1][9/952] loss = 0.69, accuracy = 50.00
[0/1][10/952] loss = 0.69, accuracy = 50.00
[0/1][11/952] loss = 0.69, accuracy = 49.48
[0/1][12/952] loss = 0.69, accuracy = 49.52
[0/1][13/952] loss = 0.69, accuracy = 49.55
[0/1][14/952] loss = 0.69, accuracy = 49.58
[0/1][15/952] loss = 0.69, accuracy = 49.61
[0/1][16/952] loss = 0.69, accuracy = 49.63
[0/1][17/952] loss = 0.69, accuracy = 49.65
[0/1][18/952] loss = 0.69, accuracy = 49.67
[0/1][19/952] loss = 0.69, accuracy = 

[0/1][184/952] loss = 0.69, accuracy = 49.99
[0/1][185/952] loss = 0.69, accuracy = 50.02
[0/1][186/952] loss = 0.69, accuracy = 49.98
[0/1][187/952] loss = 0.69, accuracy = 49.95
[0/1][188/952] loss = 0.69, accuracy = 49.95
[0/1][189/952] loss = 0.69, accuracy = 49.96
[0/1][190/952] loss = 0.69, accuracy = 49.95
[0/1][191/952] loss = 0.69, accuracy = 49.93
[0/1][192/952] loss = 0.69, accuracy = 49.95
[0/1][193/952] loss = 0.69, accuracy = 49.97
[0/1][194/952] loss = 0.69, accuracy = 49.96
[0/1][195/952] loss = 0.69, accuracy = 49.94
[0/1][196/952] loss = 0.69, accuracy = 49.98
[0/1][197/952] loss = 0.69, accuracy = 49.94
[0/1][198/952] loss = 0.69, accuracy = 49.90
[0/1][199/952] loss = 0.69, accuracy = 49.89
[0/1][200/952] loss = 0.69, accuracy = 49.88
[0/1][201/952] loss = 0.69, accuracy = 49.91
[0/1][202/952] loss = 0.69, accuracy = 49.95
[0/1][203/952] loss = 0.69, accuracy = 49.98
[0/1][204/952] loss = 0.69, accuracy = 49.95
[0/1][205/952] loss = 0.69, accuracy = 49.95
[0/1][206/

[0/1][368/952] loss = 0.69, accuracy = 49.72
[0/1][369/952] loss = 0.69, accuracy = 49.73
[0/1][370/952] loss = 0.69, accuracy = 49.72
[0/1][371/952] loss = 0.69, accuracy = 49.72
[0/1][372/952] loss = 0.69, accuracy = 49.72
[0/1][373/952] loss = 0.69, accuracy = 49.71
[0/1][374/952] loss = 0.69, accuracy = 49.72
[0/1][375/952] loss = 0.69, accuracy = 49.72
[0/1][376/952] loss = 0.69, accuracy = 49.71
[0/1][377/952] loss = 0.69, accuracy = 49.72
[0/1][378/952] loss = 0.69, accuracy = 49.72
[0/1][379/952] loss = 0.69, accuracy = 49.72
[0/1][380/952] loss = 0.69, accuracy = 49.72
[0/1][381/952] loss = 0.69, accuracy = 49.73
[0/1][382/952] loss = 0.69, accuracy = 49.74
[0/1][383/952] loss = 0.69, accuracy = 49.70
[0/1][384/952] loss = 0.69, accuracy = 49.71
[0/1][385/952] loss = 0.69, accuracy = 49.70
[0/1][386/952] loss = 0.69, accuracy = 49.71
[0/1][387/952] loss = 0.69, accuracy = 49.72
[0/1][388/952] loss = 0.69, accuracy = 49.72
[0/1][389/952] loss = 0.69, accuracy = 49.72
[0/1][390/

[0/1][552/952] loss = 0.69, accuracy = 49.83
[0/1][553/952] loss = 0.69, accuracy = 49.83
[0/1][554/952] loss = 0.69, accuracy = 49.81
[0/1][555/952] loss = 0.69, accuracy = 49.81
[0/1][556/952] loss = 0.69, accuracy = 49.81
[0/1][557/952] loss = 0.69, accuracy = 49.84
[0/1][558/952] loss = 0.69, accuracy = 49.84
[0/1][559/952] loss = 0.69, accuracy = 49.83
[0/1][560/952] loss = 0.69, accuracy = 49.83
[0/1][561/952] loss = 0.69, accuracy = 49.81
[0/1][562/952] loss = 0.69, accuracy = 49.82
[0/1][563/952] loss = 0.69, accuracy = 49.81
[0/1][564/952] loss = 0.69, accuracy = 49.80
[0/1][565/952] loss = 0.69, accuracy = 49.82
[0/1][566/952] loss = 0.69, accuracy = 49.82
[0/1][567/952] loss = 0.69, accuracy = 49.82
[0/1][568/952] loss = 0.69, accuracy = 49.82
[0/1][569/952] loss = 0.69, accuracy = 49.82
[0/1][570/952] loss = 0.69, accuracy = 49.82
[0/1][571/952] loss = 0.69, accuracy = 49.81
[0/1][572/952] loss = 0.69, accuracy = 49.80
[0/1][573/952] loss = 0.69, accuracy = 49.82
[0/1][574/

[0/1][736/952] loss = 0.69, accuracy = 49.88
[0/1][737/952] loss = 0.69, accuracy = 49.89
[0/1][738/952] loss = 0.69, accuracy = 49.88
[0/1][739/952] loss = 0.69, accuracy = 49.89
[0/1][740/952] loss = 0.69, accuracy = 49.89
[0/1][741/952] loss = 0.69, accuracy = 49.89
[0/1][742/952] loss = 0.69, accuracy = 49.89
[0/1][743/952] loss = 0.69, accuracy = 49.88
[0/1][744/952] loss = 0.69, accuracy = 49.89
[0/1][745/952] loss = 0.69, accuracy = 49.89
[0/1][746/952] loss = 0.69, accuracy = 49.89
[0/1][747/952] loss = 0.69, accuracy = 49.89
[0/1][748/952] loss = 0.69, accuracy = 49.90
[0/1][749/952] loss = 0.69, accuracy = 49.89
[0/1][750/952] loss = 0.69, accuracy = 49.90
[0/1][751/952] loss = 0.69, accuracy = 49.89
[0/1][752/952] loss = 0.69, accuracy = 49.88
[0/1][753/952] loss = 0.69, accuracy = 49.88
[0/1][754/952] loss = 0.69, accuracy = 49.87
[0/1][755/952] loss = 0.69, accuracy = 49.86
[0/1][756/952] loss = 0.69, accuracy = 49.87
[0/1][757/952] loss = 0.69, accuracy = 49.88
[0/1][758/

[0/1][920/952] loss = 0.69, accuracy = 49.84
[0/1][921/952] loss = 0.69, accuracy = 49.84
[0/1][922/952] loss = 0.69, accuracy = 49.84
[0/1][923/952] loss = 0.69, accuracy = 49.84
[0/1][924/952] loss = 0.69, accuracy = 49.83
[0/1][925/952] loss = 0.69, accuracy = 49.85
[0/1][926/952] loss = 0.69, accuracy = 49.85
[0/1][927/952] loss = 0.69, accuracy = 49.84
[0/1][928/952] loss = 0.69, accuracy = 49.84
[0/1][929/952] loss = 0.69, accuracy = 49.85
[0/1][930/952] loss = 0.69, accuracy = 49.85
[0/1][931/952] loss = 0.69, accuracy = 49.86
[0/1][932/952] loss = 0.69, accuracy = 49.86
[0/1][933/952] loss = 0.69, accuracy = 49.85
[0/1][934/952] loss = 0.69, accuracy = 49.84
[0/1][935/952] loss = 0.69, accuracy = 49.84
[0/1][936/952] loss = 0.69, accuracy = 49.84
[0/1][937/952] loss = 0.69, accuracy = 49.85
[0/1][938/952] loss = 0.69, accuracy = 49.85
[0/1][939/952] loss = 0.69, accuracy = 49.85
[0/1][940/952] loss = 0.69, accuracy = 49.85
[0/1][941/952] loss = 0.69, accuracy = 49.85
[0/1][942/

In [14]:
# Ascertain best results

df.at[df_idx,'Precision'] = round(df_pr[df_pr['Accuracy']==df_pr['Accuracy'].max()].Precision.values[0],4)
df.at[df_idx,'Recall'] = round(df_pr[df_pr['Accuracy']==df_pr['Accuracy'].max()].Recall.values[0],4)


In [ ]:
df.to_pickle(summary_file)

# Evaluate Attack Nets
How well the trained attack models classify a sample as in or out of a target model's training dataset, and how performance is affected by target hyperparameters and which models attack which targets.

In [ ]:
df['# speakers'] =df['# speakers'].astype(float)
df['Training epochs'] =df['Training epochs'].astype(float)
df['Attack type'] =df['Attack type'].astype(float)

#style table
import seaborn as sns

cg = sns.light_palette("green", as_cmap=True)
cm = sns.light_palette("magenta", as_cmap=True)
bl = sns.light_palette("blue", as_cmap=True)
orr = sns.light_palette("orange", as_cmap=True)
gr = sns.light_palette("gray", as_cmap=True)

# df.style.bar(subset=['Train accuracy', 'Test accuracy'], align='mid', color=['#d65f5f', '#5fba7d'])
s = df.style.\
    background_gradient(cmap=cg,subset=['Train accuracy', 'Test accuracy']).\
    background_gradient(cmap=bl,subset=['Precision', 'Recall']).\
    background_gradient(cmap=orr,subset=['Training epochs']).\
    background_gradient(cmap=gr,subset=['Attack type']).\
    background_gradient(cmap=cm,subset=['# speakers']).\
    format({"Train accuracy": "{:.2%}","Test accuracy": "{:.2%}"}).\
    hide_index().\
    set_properties(**{'font-size': "16pt",'column-size':"24pt",'width': '100px'})

s

In [ ]:
#old, for reference for now:

# do this for 10 & 100 speakers
# .2 S & 3 S
#sufficient training and over-training

#manual data: 

#Attack 1:
df.loc[len(df)] = ['MFCC',25,69.0,.9994,.9632,1,0.89,0.90] 
df.loc[len(df)] = ['MFCC',25,255.0,.9961,.9443,1,0.88,0.91] 
df.loc[len(df)] = ['STFT',25,69.0,0.9989,0.9451,1,0.89,0.92] 
df.loc[len(df)] = ['STFT',25,255.0,0.9958,0.9181,1,0.85,0.86] 

#Attack 3 w/max data: 
df.loc[len(df)] = ['STFT',25,139.0,.9985,.9073,3,.81,.90] 
df.loc[len(df)] = ['STFT',50,511.0,.9942,.9057,3,.84,.87] 
df.loc[len(df)] = ['MFCC',25,139.0,.9969,.9136,3,.82,.92] 
df.loc[len(df)] = ['MFCC',25,511.0,.9960,.9321,3,0.83,0.93]


#Attack 3 on Attack1 models:
df.loc[len(df)] = ['MFCC',25,69.0,.9994,.9632,3,0.84,0.95] 
df.loc[len(df)] = ['MFCC',25,255.0,.9961,.9443,3,0.84,0.94] 
df.loc[len(df)] = ['STFT',25,69.0,0.9989,0.9451,3,0.81,0.97] 
df.loc[len(df)] = ['STFT',25,255.0,0.9958,0.9181,3,0.81,0.90] 

df['# speakers'] =df['# speakers'].astype(float)
df['Training epochs'] =df['Training epochs'].astype(float)
df['Attack type'] =df['Attack type'].astype(float)

#style table
import seaborn as sns

cg = sns.light_palette("green", as_cmap=True)
cm = sns.light_palette("magenta", as_cmap=True)
bl = sns.light_palette("blue", as_cmap=True)
orr = sns.light_palette("orange", as_cmap=True)
gr = sns.light_palette("gray", as_cmap=True)

# df.style.bar(subset=['Train accuracy', 'Test accuracy'], align='mid', color=['#d65f5f', '#5fba7d'])
s = df.style.\
    background_gradient(cmap=cg,subset=['Train accuracy', 'Test accuracy']).\
    background_gradient(cmap=bl,subset=['Precision', 'Recall']).\
    background_gradient(cmap=orr,subset=['Training epochs']).\
    background_gradient(cmap=gr,subset=['Attack type']).\
    background_gradient(cmap=cm,subset=['# speakers']).\
    format({"Train accuracy": "{:.2%}","Test accuracy": "{:.2%}"}).\
    hide_index().\
    set_properties(**{'font-size': "16pt",'column-size':"24pt",'width': '100px'})

s